In [17]:
import os, glob, math
path = 'emails/bare/part1'
# os.path = returns current directory
path = os.path.join(os.getcwd(), path)

class email:
    # represents each email 
    def __init__(self):
        self.classType = 'undefined'
        self.words = []
        self.vector = []
        self.predict = 'undefined'
    def openFile(self, classType, filePath):
        self.classType = classType
        with open(filePath) as myFile:
            data = myFile.read()
            words = set()
            words = data.split()
            self.words = set(words)
    def createVector(self, vocab):
        for word in vocab:
            if(word in self.words):
                self.vector.append(1)
            else:
                self.vector.append(0)

class dictionary:
    # word dictionary where words are stored and 
    def __init__(self):
#         self.spamWords = []
#         self.hamWords = []
        self.words = []
        self.spamCount = 0
        self.hamCount = 0
    def update(self, mail):
        tempWordBag = []
        for word in self.words:
            tempWordBag.append(word[0])
        mailWords = mail.words
        if(mail.classType == 'ham'):
            self.hamCount += 1
        else:
            self.spamCount += 1
        for mword in mailWords:
            if(mword in tempWordBag):
                index = tempWordBag.index(mword)
                sword = self.words[index]
                if(mail.classType == 'ham'):
                    sword[1] += 1
                else:
                    sword[2] += 1
            else:
                sword = [mword, 0, 0]
                if(mail.classType == 'ham'):
                    sword[1] += 1
                else:
                    sword[2] += 1
                self.words.append(sword)
                
#         if(mail.classType == 'ham'):
#             self.hamCount += 1
#             for word in self.hamWords:
#                 tempWordBag.append(word[0])
#             mailWords = mail.words
#             for mword in mailWords:
#                 if(mword in tempWordBag):
#                     index = tempWordBag.index(mword)
#                     sword = self.hamWords[index]
#                     sword[1] += 1
#                 else:
#                     sword = [mword, 1]
#                     self.hamWords.append(sword)
#         else:
#             self.spamCount += 1
#             for word in self.spamWords:
#                 tempWordBag.append(word[0])
#             mailWords = mail.words
#             for mword in mailWords:
#                 if(mword in tempWordBag):
#                     index = tempWordBag.index(mword)
#                     sword = self.spamWords[index]
#                     sword[1] += 1
#                 else:
#                     sword = [mword, 1]
#                     self.spamWords.append(sword)
    def display(self):
        print('spam count: ', self.spamCount)
        print('ham count: ', self.hamCount)
        for word in self.words:
            print(word)
#         print('spam words')
#         for word in self.spamWords:
#             print(word)
#         print('ham words')
#         for word in self.hamWords:
#             print(word)

wordDict = dictionary()
mailList = []

for filePath in glob.glob(os.path.join(path, '*.txt')):
    mail = email()
    if('spmsga' in filePath):
        mail.openFile('spam', filePath)
    else:
        mail.openFile('ham', filePath)
    mailList.append(mail)
    wordDict.update(mail)



In [18]:
class mutualInformation:
    def __init__(self, countHam, countSpam):
        self.probHam = 1.0 * countHam / (countHam+countSpam)
        self.probSpam = 1.0 * countSpam / (countHam+countSpam)
        self.words = []
    def probOfWord(self, words):
        countH = 0
        countS = 0
        for word in words:
            countH += word[1]
            countS += word[2]
        count = countH+countS
        for tempWord in words:
#             print(tempWord)
            # probability of a word appearing
            wordprob = 1.0 * (tempWord[1]+tempWord[2])/ count
            # calculate the probability of word given a class(ham)
            hwordprob = 1.0 * self.probHam * (1.0 * tempWord[1]/countH)
            # calculate the probability of word given a class(ham), P(X,C)*log(P(X,C)/(P(X)*P(C)))
            if(hwordprob == 0):
                htemp = 0
            else:
                htemp = hwordprob * math.log(hwordprob/(wordprob*self.probHam))
            # calculate the probability of word given a class(ham)
            swordprob = 1.0 * self.probSpam * (1.0 * tempWord[2]/count)
            # calculate the probability of word given a class(ham), P(X,C)*log(P(X,C)/(P(X)*P(C)))
            if(swordprob == 0):
                stemp = 0
            else:
                stemp = swordprob * math.log(swordprob/(wordprob*self.probSpam))
            temp = htemp + stemp
#             print(temp)
            word = [tempWord[0], temp]
            self.words.append(word)
    def getmi(self, word):
        # function to return only the miscore
        return word[1]
    def sortWords(self):
        # sort words according to miscore, descending
        self.words.sort(key = self.getmi, reverse = True)
    def getWords(self, number):
        vocab = []
        for index in range(number):
            word = self.words[index]
            vocab.append(word[0])
        return vocab
        

    
mi = mutualInformation(wordDict.hamCount, wordDict.spamCount)
print('probability of ham: %f' %mi.probHam)
print('probability of spam: %f' %mi.probSpam)
# calculate probability of a word
words = wordDict.words

mi.probOfWord(words)
mi.sortWords()
# for word in mi.words:
#     print(word[0], '%.5f' %word[1])
vocab = mi.getWords(50)
for voc in vocab:
    print(voc)
    

probability of ham: 0.833910
probability of spam: 0.166090
linguistics
language
university
languages
english
(
edu
department
linguistic
linguist
linguists
)
i
@
who
but
some
which
discussion
query
research
syntax
was
he
de
;
1992
theory
speakers
modern
grammar
data
analysis
issue
second
rather
his
both
published
whether
think
as
theoretical
chomsky
far
general
literature
two
in
would


In [20]:
# temp = mailList[0]
# temp.createVector(vocab)
# print(temp.vector)
for mail in mailList:
    mail.createVector(vocab)
    print(mail.vector)

[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1]
[0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
[0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0